In [3]:
import torch
import pyaudio
import wave
import os
import model.AudioUtil as AudioUtil
from model.SoundDS import SoundDS
# from model.AudioClassifier import AudioClassifier

print(torch.__version__)
model = torch.load('data/models/V8_model_fullV2.pth')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
model.eval()

CHUNK = 1050
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

2.3.0+cu118
Using device: cuda:0


In [4]:
def record_audio(duration=2):
    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    try:
        # Calculate the correct number of iterations to cover the duration
        num_frames = int((RATE / CHUNK) * duration)
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
    return frames

def save_temp_audio(frames, filename="temp_audio.wav"):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def process_audio_file(filename):
    class_names = {0: 'Noise', 1: 'Knock', 2: 'kn_se'} 
    aud = AudioUtil.open(filename)
    # os.remove(filename)
    # reaud = AudioUtil.resample(aud, RATE)
    # rechan = AudioUtil.rechannel(reaud, CHANNELS)
    # dur_aud = AudioUtil.pad_trunc(rechan, 2000)
    rechan = AudioUtil.rechannel(aud, 2)
    sgram = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
    # sgram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram = sgram.to(device)
    print(type(sgram))

    # sgram_tensor = torch.tensor(sgram).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        inputs = sgram

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        outputs = model(inputs.unsqueeze(0))

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs, 1)
        # Convert predictions and actual labels to class names
        predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction][0]
        print(outputs)
    # print(prediction.item())
    print(predicted_classes)
    # if prediction > 0.5:
    #     print("Klopfen erkannt!")

In [6]:
# for file in os.listdir("data/validation"):
#     print(f"Processing {file}")
#     process_audio_file(f"data/validation/{file}")
process_audio_file("data/validation/noise_81_var4.wav")

LibsndfileError: Error opening 'data/validation/noise_81_var4.wav': System error.

In [8]:
print("Starte Klopfen-Erkennung...")
while True:
    frames = record_audio()
    save_temp_audio(frames)
    process_audio_file("temp_audio.wav")

Starte Klopfen-Erkennung...


OSError: [Errno -9996] Invalid input device (no default output device)

In [4]:

class_names = {0: 'Noise', 1: 'Knock', 2:'Kn_se'}  # Mapping class indices to class names

# ANSI escape codes for colors
RED = '\033[91m'   # Red text
RESET = '\033[0m'  # Reset to default color

# Disable gradient updates
with torch.no_grad():
    filename = "knock_1.wav"
    aud = AudioUtil.open(filename)
    rechan = AudioUtil.rechannel(aud, 2)
    inputs = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
    inputs = inputs.to(device)

    ds = SoundDS('')


    # Normalize the inputs
    inputs_m, inputs_s = inputs.mean(), inputs.std()
    inputs = (inputs - inputs_m) / inputs_s

    # Get predictions
    outputs = model(inputs.unsqueeze(0))

    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs, 1)
    # Convert predictions and actual labels to class names
    predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
    row_len = 50
    # Prepare to print 10 predictions and actuals per line
    for i in range(0, len(predicted_classes), row_len):
        pred_slice = predicted_classes[i:i+row_len]
        print('Predictions: ', ' '.join(pred_slice))

Predictions:  Knock


In [5]:
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0
    class_names = {0: 'Noise', 1: 'Knock', 2:'Kn_se'}  # Mapping class indices to class names

    # ANSI escape codes for colors
    RED = '\033[91m'   # Red text
    RESET = '\033[0m'  # Reset to default color

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs, 1)
            # Convert predictions and actual labels to class names
            predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
            actual_classes = [class_names.get(l.item(), l.item()) for l in labels]
            row_len = 50
            # Prepare to print 10 predictions and actuals per line
            for i in range(0, len(predicted_classes), row_len):
                pred_slice = predicted_classes[i:i+row_len]
                actual_slice = actual_classes[i:i+row_len]
                pred_str = ', '.join([f'{RED if pred != actual else RESET}{pred}{RESET}' for pred, actual in zip(pred_slice, actual_slice)])
                actual_str = ', '.join([f'{RED if pred != actual else RESET}{actual}{RESET}' for pred, actual in zip(pred_slice, actual_slice)])
                print(f'Predicted: {pred_str}\nActual:    {actual_str}\n')

            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [9]:
%load_ext autoreload
%autoreload 2

In [11]:
from model.SoundDS import SoundDS
from torch.utils.data import DataLoader
single_ds = SoundDS('data/validation')
single_dl = DataLoader(single_ds)

inference(model, single_dl)

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Actual:    Knock

Predicted: Knock
Act